In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-0.7576  0.4215 -0.4827 -1.1198  0.3056
[torch.FloatTensor of size 1x5]



## N-Gram Language Modeling

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [6]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [7]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [8]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [12]:
for epoch in range(20):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 524.8329
[torch.FloatTensor of size 1]
, 
 522.3813
[torch.FloatTensor of size 1]
, 
 519.9492
[torch.FloatTensor of size 1]
, 
 517.5349
[torch.FloatTensor of size 1]
, 
 515.1374
[torch.FloatTensor of size 1]
, 
 512.7549
[torch.FloatTensor of size 1]
, 
 510.3875
[torch.FloatTensor of size 1]
, 
 508.0355
[torch.FloatTensor of size 1]
, 
 505.6968
[torch.FloatTensor of size 1]
, 
 503.3706
[torch.FloatTensor of size 1]
, 
 501.0567
[torch.FloatTensor of size 1]
, 
 498.7539
[torch.FloatTensor of size 1]
, 
 496.4609
[torch.FloatTensor of size 1]
, 
 494.1769
[torch.FloatTensor of size 1]
, 
 491.8998
[torch.FloatTensor of size 1]
, 
 489.6313
[torch.FloatTensor of size 1]
, 
 487.3702
[torch.FloatTensor of size 1]
, 
 485.1158
[torch.FloatTensor of size 1]
, 
 482.8683
[torch.FloatTensor of size 1]
, 
 480.6266
[torch.FloatTensor of size 1]
, 
 478.3906
[torch.FloatTensor of size 1]
, 
 476.1589
[torch.FloatTensor of size 1]
, 
 473.9325
[torch.FloatTensor of size 1]
, 
 471.7101

## Continuous Bag of Words

In [19]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 15
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}

In [20]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [25]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2*context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [22]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


make_context_vector(data[0][0], word_to_ix)

Variable containing:
 13
  7
 18
 42
[torch.LongTensor of size 4]

In [26]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [27]:
for epoch in range(20):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_var = make_context_vector(context, word_to_ix)
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 230.1675
[torch.FloatTensor of size 1]
, 
 228.4702
[torch.FloatTensor of size 1]
, 
 226.7866
[torch.FloatTensor of size 1]
, 
 225.1132
[torch.FloatTensor of size 1]
, 
 223.4518
[torch.FloatTensor of size 1]
, 
 221.8006
[torch.FloatTensor of size 1]
, 
 220.1565
[torch.FloatTensor of size 1]
, 
 218.5209
[torch.FloatTensor of size 1]
, 
 216.8920
[torch.FloatTensor of size 1]
, 
 215.2690
[torch.FloatTensor of size 1]
, 
 213.6521
[torch.FloatTensor of size 1]
, 
 212.0411
[torch.FloatTensor of size 1]
, 
 210.4346
[torch.FloatTensor of size 1]
, 
 208.8313
[torch.FloatTensor of size 1]
, 
 207.2342
[torch.FloatTensor of size 1]
, 
 205.6399
[torch.FloatTensor of size 1]
, 
 204.0485
[torch.FloatTensor of size 1]
, 
 202.4586
[torch.FloatTensor of size 1]
, 
 200.8690
[torch.FloatTensor of size 1]
, 
 199.2776
[torch.FloatTensor of size 1]
]
